In [154]:
import ee
import geemap
import numpy as np

In [155]:
#Create Map and center on Manauas EC Flux Tower coordinates
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
brma2_point = ee.Geometry.Point(-60.20929722, -2.609097222)
brma2_region = ee.Geometry.Rectangle(-60.50, -2.85, -60.00, -2.35)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [156]:
Map.addLayer(brma2_region, {}, 'Manauas Region')

In [157]:
sent2 = ee.ImageCollection('COPERNICUS/S2')

In [158]:
#Load s2 data for July 2016
s2 = (ee.ImageCollection('COPERNICUS/S2')
      .filterDate('2016-07-01', '2016-07-31')
     .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
     .map(lambda image : image.clip(brma2_region)))
visParams = {'bands' : ['B4', 'B3', 'B2'], 'max': 3000}
Map.addLayer(s2, visParams, "July 2016 Raw")

In [ ]:
#Calculate NDRE and get median image for July 2016
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)
ndre_params = ({'min': 0.0,'max': 1.0, 'palette' : 
               ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']})

In [159]:
#User Dr. Wu's ee_to_numpy function
#Only accepts an image, not a collection
jul2016 = (s2.filterDate('2016-07-01', '2016-07-31')
           .map(addNDRE).select('NDRE'))
jul16_med = ee.Image(jul2016.median())
array = geemap.ee_to_numpy(jul16_med, region = brma2_point)
print(array)

In [162]:
#Dr. Wu's function broken down
bands = jul16_med.bandNames().getInfo()
print(str('band names: ') + str(bands))

band_arrs = jul16_med.sampleRectangle(region=brma2_point)
band_values = []
for band in bands:
    band_arr = band_arrs.get(band).getInfo()
    band_value = np.array(band_arr)
    band_values.append(band_value)
    
image = np.dstack(band_values)
print(image)

band names: ['NDRE']
[[[0.53382969]]]


In [150]:
#List sentinel2 metadata properties to see what I can select
properties = s2.first().propertyNames().getInfo()
print(properties)

['system:footprint', 'DATATAKE_IDENTIFIER', 'SPACECRAFT_NAME', 'FORMAT_CORRECTNESS_FLAG', 'IERS_BULLETIN_FILENAME', 'system:id', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A', 'MEAN_SOLAR_AZIMUTH_ANGLE', 'SOLAR_IRRADIANCE_B12', 'system:version', 'SOLAR_IRRADIANCE_B10', 'SOLAR_IRRADIANCE_B11', 'GENERATION_TIME', 'SOLAR_IRRADIANCE_B8A', 'PRODUCT_URI', 'SENSOR_QUALITY_FLAG', 'CLOUD_COVERAGE_ASSESSMENT', 'system:time_end', 'system:time_start', 'DATASTRIP_ID', 'PROCESSING_BASELINE', 'SENSING_ORBIT_NUMBER', 'GEOMETRIC_QUALITY_FLAG', 'SENSING_ORBIT_DIRECTION', 'GRANULE_ID', 'REFLECTANCE_CONVERSION_CORRECTION', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8', 'DATATAKE_TYPE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B6', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B7', 'RADIOMETRIC_QUALITY_FLAG', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B4', 'MEAN_INCIDENCE_ZENITH_ANGLE_B1', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B5', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B2', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B3', 'MEAN_INCIDENCE_ZENITH_ANGLE_B5', 'MEAN_I